In [152]:
import os 
from glob import glob 
import pandas as pd 
import numpy as np 

In [153]:
def parse_line(raw_line, delimiter=' '):
    line = []
    for i in raw_line.strip().split(delimiter):
        if i != '':
            line.append(i)
    return line


In [154]:
def find_finished_jobs( output_folder ):
    return_list = []
    job_dirs = [ os.path.abspath(os.path.join(output_folder, d)) for d in os.listdir(output_folder) if os.path.isdir( os.path.join(output_folder, d) )]
    for job_dir in job_dirs:
        completed_struct_dirs = []
        for potential_struct_dir in sorted([ os.path.abspath(os.path.join(job_dir, d)) for d in os.listdir(job_dir) if os.path.isdir( os.path.join(job_dir, d) )]):
            completed_struct_dirs.append( potential_struct_dir )
        return_list.append(job_dir)

    return return_list

In [155]:
def get_ddg(job):
    ddg_file = glob(job + '/*ddg')[0]
    case_name = os.path.basename(job)
    with open(ddg_file) as f:
        data = f.readlines()
    for i,line in enumerate(data):
        data[i] = parse_line(line)
    round_num = int(len(data)/2)
    MUT = float(data[round_num*2 -1][3])
    WT = float(data[round_num - 1][3])
    ddg = MUT -WT
    return case_name, WT, MUT, ddg 

In [156]:
jobs = find_finished_jobs('./output_saturation')

In [157]:
df = pd.DataFrame(columns=['case name', 'WT', 'MUT', 'ddG'])
for job in jobs:
    case_name, WT, MUT, ddg = get_ddg(job)
    job_dict = {}
    job_dict['case name'] = case_name
    job_dict['WT'] = WT
    job_dict['MUT'] = MUT
    job_dict['ddG'] = ddg
    df = df.append(job_dict,ignore_index=True)

In [149]:
df = df.sort_values(['case name'])

In [151]:
df.to_csv('summary_casename.csv')